In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.447), (0.247, 0.243, 0.262))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
DenseNet = models.densenet121(pretrained = True)
DenseNet.classifier = nn.Linear(1024, 10)
DenseNet.to(device)
print(DenseNet)

/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
# optimizer
optimizer = optim.SGD(DenseNet.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [6]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [7]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [8]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [9]:
import torch
import torch.nn.functional as F

def majority_vote_prediction(model, images, transformations, device):
    """
    Given a batch of images and a list of transformations,
    compute the model's predictions on:
        1) the original images,
        2) each transformed version,
    then take a majority vote across all predictions for each sample.
    
    Returns a tensor of shape [batch_size] with the final voted class.
    """
    model.eval()

    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape [batch_size]

    # 2) Collect predictions in a list
    all_preds = [original_preds]

    # 3) For each metamorphic transform
    for tf in transformations:
        # apply transform on CPU, then move back to device
        x_tf = tf(images.cpu()).to(device)

        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)

        all_preds.append(preds_tf)

    # 4) Stack them into shape [num_transforms+1, batch_size]
    stacked_preds = torch.stack(all_preds, dim=0)

    # 5) Majority vote across the first dimension
    voted_preds = stacked_preds.mode(dim=0).values  # shape [batch_size]
    return voted_preds

In [10]:
from tqdm import tqdm

def test_with_majority_vote(model, loader, transformations, device):
    """
    Evaluates the model on 'loader' using majority-voted predictions (original + transforms).
    Prints per-batch accuracy and returns the final overall accuracy.
    """
    model.eval()
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)

        # Get majority-voted predictions
        voted_preds = majority_vote_prediction(model, images, transformations, device)

        # Compare to ground-truth
        batch_correct = (voted_preds == labels).sum().item()
        batch_size = labels.size(0)

        correct += batch_correct
        total += batch_size

        # Print a line for each batch
        print(f"Batch {i} | MajVot Accuracy: {100.0 * batch_correct / batch_size:.2f}% | Batch Size: {batch_size}")

    # Final overall accuracy
    overall_accuracy = 100.0 * correct / total
    print(f"\nFinal MajVot Accuracy (across all test samples): {overall_accuracy:.2f}%")
    return overall_accuracy

In [11]:
majvot_acc = test_with_majority_vote(DenseNet, testloader, transformations, device)
print(f"Majority Vote final accuracy: {majvot_acc:.2f}%")

  0%|▎                                                                                     | 2/625 [00:00<03:44,  2.77it/s]

Batch 0 | MajVot Accuracy: 12.50% | Batch Size: 16
Batch 1 | MajVot Accuracy: 6.25% | Batch Size: 16


  1%|▌                                                                                     | 4/625 [00:01<02:31,  4.09it/s]

Batch 2 | MajVot Accuracy: 0.00% | Batch Size: 16
Batch 3 | MajVot Accuracy: 0.00% | Batch Size: 16


  1%|▊                                                                                     | 6/625 [00:01<02:14,  4.60it/s]

...

Batch 620 | MajVot Accuracy: 6.25% | Batch Size: 16
Batch 621 | MajVot Accuracy: 12.50% | Batch Size: 16


100%|███████████████████████████████████████████████████████████████████████████████████▊| 624/625 [01:48<00:00,  5.64it/s]

Batch 622 | MajVot Accuracy: 6.25% | Batch Size: 16
Batch 623 | MajVot Accuracy: 6.25% | Batch Size: 16


100%|████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:48<00:00,  5.78it/s]

Batch 624 | MajVot Accuracy: 0.00% | Batch Size: 16

Final MajVot Accuracy (across all test samples): 10.38%
Majority Vote final accuracy: 10.38%
